# Evaluate the BERT model

## 1. Get the model

## 2. Get the data

In [ ]:
def load_pickle_data(filename):
    with open(filename, "rb") as load_file:
        return pickle.load(load_file)

In [ ]:
eval_data = load_pickle_data("BERT_data_final/_evaluation_data.pickle")

## 3. Define an evaluator

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
class SenseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def evaluate_BERT(model, eval_data):
    print(f"Model evaluation started!")
    test_acc = {}
    model.eval()

    for lemma in eval_data.keys():
        print(f"Evaluating model for lemma {lemma}...")
        with torch.no_grad():
            accuracy_sum = 0
            nbr_test_samples = 0
            for dict_entry in eval_data[lemma]:
                nbr_test_samples += 1
                eval_encodings = tokenizer(dict_entry["X_data_1"], dict_entry["X_data_2"], truncation=True, padding=True, return_tensors="pt")
                outputs = model(**eval_encodings)
                #print(f"output: {output.numpy().flatten()}")
                print(f"output: {outputs.logits}")
                correct_index_guess = np.argmax(outputs.logits.numpy().flatten())
                #print(f"correct index guess: {correct_index_guess}")
                #loss = loss_fun(output, by.type(torch.FloatTensor))
                #loss_sum += loss.item()
                accuracy = [1 if correct_index_guess==dict_entry["y_data"][0] else 0]
                #print(f"accuracy: {accuracy}")
                accuracy_sum += accuracy[0]
        #test_loss = loss_sum/(nbr_test_batches*batch_size)
        #print(h)
        test_acc[lemma] = accuracy_sum/nbr_test_samples
    return test_acc

In [ ]:
def evaluate_model_per_lemma(model, eval_data):
    print(f"Model evaluation started!")
    test_acc = {}
    model.eval()
    for lemma in eval_data.keys():
        print(f"Evaluating model for lemma {lemma}...")
        with torch.no_grad():
            accuracy_sum = 0
            nbr_test_samples = 0
            for dict_entry in eval_data[lemma]:
                nbr_test_samples += 1
                output = model.forward(torch.LongTensor(dict_entry["X_data"]).to(device), torch.BoolTensor(dict_entry["mask_data"]).to(device))
                #print(f"output: {output.numpy().flatten()}")
                correct_index_guess = np.argmax(output.numpy().flatten())
                #print(f"correct index guess: {correct_index_guess}")
                #loss = loss_fun(output, by.type(torch.FloatTensor))
                #loss_sum += loss.item()
                accuracy = [1 if correct_index_guess==dict_entry["y_data"][0] else 0]
                #print(f"accuracy: {accuracy}")
                accuracy_sum += accuracy[0]
        #test_loss = loss_sum/(nbr_test_batches*batch_size)
        #print(h)
        test_acc[lemma] = accuracy_sum/nbr_test_samples
    return test_acc

## 4. Evaluate!